## Loading and cleaning

In [10]:
import sys
sys.path.insert(0, '../repository')

In [11]:
import database
import pandas as pd
import numpy as np
import glob
import re

In [12]:
conn = database.DatabaseConnection(db_name='doing-bussiness').connection

In [13]:
f = open("../repository/DUMP/ISO CODE.csv", "r")
iso_codes = pd.read_csv(f)
iso_codes['id'] = pd.Series(range(len(iso_codes)), name='id')

f = open("../repository/RAW/WB - Doing Business.csv", mode="r", encoding="utf-8")
df = pd.read_csv(f, sep=";", index_col=False)
df = df.dropna(how="all").convert_dtypes()

In [14]:
df.columns = [re.sub(r"\(.*\)|'", "", c).strip() for c in df.columns]
df.columns = [re.sub(r"-| ", "_", c).lower() for c in df.columns]

In [15]:
df = df.merge(iso_codes.iloc[:, [-1, 0, 2]], left_on="country_code", right_on="alpha-3")
df = df.drop(axis=1, columns=["economy", "name", "alpha-3", "region"])
df = df.rename(columns={"id": "country_id", "db_year": "year"})
df = df[["country_id", "year", *df.columns]]
df = df.drop(columns=["country_code"])
df = df.loc[:, ~df.columns.duplicated()].copy()
list(df.columns)

['country_id',
 'year',
 'income_group',
 'ease_of_doing_business_rank',
 'ease_of_doing_business_score',
 'rank_starting_a_business',
 'score_starting_a_business',
 'procedures___men',
 'score_procedures___men',
 'time___men',
 'score_time___men',
 'cost___men',
 'score_cost___men',
 'procedures___women',
 'score_procedures___women',
 'time___women',
 'score_time___women',
 'cost___women',
 'score_cost___women',
 'paid_in_minimum_capital',
 'score_paid_in_minimum_capital',
 'rank_dealing_with_construction_permits',
 'score_dealing_with_construction_permits',
 'procedures',
 'score_procedures',
 'time',
 'score_time',
 'cost',
 'score_cost',
 'building_quality_control_index',
 'score_building_quality_control_index',
 'quality_of_building_regulations_index',
 'quality_control_before_construction_index',
 'quality_control_during_construction_index',
 'quality_control_after_construction_index',
 'liability_and_insurance_regimes_index',
 'professional_certifications_index',
 'rank_getting_

In [16]:
# df.to_csv('../repository/DUMP/Doing Business/DB - Freedom.csv', sep=';', index=False, decimal='.')
df.to_sql('freedom', con=conn, if_exists='replace', index_label='id')

59